<a href="https://colab.research.google.com/github/sahilfaizal01/Two-Stage-Brain-Tumor-Recognition-and-Identification/blob/main/resunet_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import cv2
from skimage import io
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, normalize
import os
import glob
import random
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
brain_df = pd.read_csv('/content/drive/MyDrive/Brain-Tumor-Segmentation/Brain_MRI/'+'data_mask.csv')

In [4]:
brain_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3929 entries, 0 to 3928
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   patient_id  3929 non-null   object
 1   image_path  3929 non-null   object
 2   mask_path   3929 non-null   object
 3   mask        3929 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 122.9+ KB


In [5]:
brain_df.head(50)

,patient_id,image_path,mask_path,mask
0,TCGA_CS_5395_19981004,TCGA_CS_5395_19981004/TCGA_CS_5395_19981004_1.tif,TCGA_CS_5395_19981004/TCGA_CS_5395_19981004_1_...,0
1,TCGA_CS_5395_19981004,TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_1.tif,TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_1_...,0
2,TCGA_CS_5395_19981004,TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_1.tif,TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_1_...,0
3,TCGA_CS_5395_19981004,TCGA_CS_4943_20000902/TCGA_CS_4943_20000902_1.tif,TCGA_CS_4943_20000902/TCGA_CS_4943_20000902_1_...,0
4,TCGA_CS_5395_19981004,TCGA_CS_5396_20010302/TCGA_CS_5396_20010302_1.tif,TCGA_CS_5396_20010302/TCGA_CS_5396_20010302_1_...,0
5,TCGA_CS_5395_19981004,TCGA_CS_5393_19990606/TCGA_CS_5393_19990606_1.tif,TCGA_CS_5393_19990606/TCGA_CS_5393_19990606_1_...,0
6,TCGA_CS_5395_19981004,TCGA_CS_4942_19970222/TCGA_CS_4942_19970222_1.tif,TCGA_CS_4942_19970222/TCGA_CS_4942_19970222_1_...,0
7,TCGA_CS_5395_19981004,TCGA_CS_5397_20010315/TCGA_CS_5397_20010315_1.tif,TCGA_CS_5397_20010315/TCGA_CS_5397_20010315_1_...,0
8,TCGA_CS_5395_19981004,TCGA_CS_6188_20010812/TCGA_CS_6188_20010812_1.tif,TCGA_CS_6188_20010812/TCGA_CS_6188_20010812_1_...,0
9,TCGA_CS_5395_19981004,TCGA_CS_6666_20011109/TCGA_CS_6666_20011109_1.tif,TCGA_CS_6666_20011109/TCGA_CS_6666_20011109_1_...,0


In [6]:
brain_df.mask_path[1] # Path to the brain MRI image

'TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_1_mask.tif'

In [7]:
brain_df.image_path[1] # Path to the segmentation mask

'TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_1.tif'

# **Building a Segmentation Model**

In [8]:
# Get the dataframe containing MRIs which have masks associated with them.
brain_df_mask = brain_df[brain_df['mask'] == 1]
brain_df_mask.shape

(1373, 4)

In [9]:
# split the data into train and test data

from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(brain_df_mask, test_size=0.15)
X_test, X_val = train_test_split(X_val, test_size=0.5)

In [10]:
X_train.image_path

3018    TCGA_FG_7643_20021104/TCGA_FG_7643_20021104_31...
2410    TCGA_DU_7014_19860618/TCGA_DU_7014_19860618_23...
2818    TCGA_FG_7637_20000922/TCGA_FG_7637_20000922_28...
1851    TCGA_HT_8114_19981030/TCGA_HT_8114_19981030_17...
1081    TCGA_HT_8114_19981030/TCGA_HT_8114_19981030_10...
                              ...                        
2188    TCGA_DU_8164_19970111/TCGA_DU_8164_19970111_20...
2815    TCGA_FG_6690_20020226/TCGA_FG_6690_20020226_28...
996     TCGA_CS_4942_19970222/TCGA_CS_4942_19970222_10...
1702    TCGA_FG_5964_20010511/TCGA_FG_5964_20010511_16...
1400    TCGA_HT_7693_19950520/TCGA_HT_7693_19950520_13...
Name: image_path, Length: 1167, dtype: object

In [11]:
# create separate list for imageId, classId to pass into the generator

train_ids = list('/drive/MyDrive/Brain-Tumor-Segmentation/Brain_MRI/'+X_train.image_path)
train_mask = list('/drive/MyDrive/Brain-Tumor-Segmentation/Brain_MRI/'+X_train.mask_path)

val_ids = list('/drive/MyDrive/Brain-Tumor-Segmentation/Brain_MRI/'+X_val.image_path)
val_mask= list('/drive/MyDrive/Brain-Tumor-Segmentation/Brain_MRI/'+X_val.mask_path)

In [12]:
train_ids[0]

'/drive/MyDrive/Brain-Tumor-Segmentation/Brain_MRI/TCGA_FG_7643_20021104/TCGA_FG_7643_20021104_31.tif'

In [24]:
# Utilities file contains the code for custom loss function and custom data generator
from utilities import DataGenerator

# create image generators
training_generator = DataGenerator(train_ids,train_mask)
validation_generator = DataGenerator(val_ids,val_mask)

In [25]:
def resblock(X, f):
  # make a copy of input
    X_copy = X
  # main path
    X = Conv2D(f, kernel_size = (1,1) ,strides = (1,1),kernel_initializer ='he_normal')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Conv2D(f, kernel_size = (3,3), strides =(1,1), padding = 'same', kernel_initializer ='he_normal')(X)
    X = BatchNormalization()(X)
  # Short path
    X_copy = Conv2D(f, kernel_size = (1,1), strides =(1,1), kernel_initializer ='he_normal')(X_copy)
    X_copy = BatchNormalization()(X_copy)
  # Adding the output from main path and short path together
    X = Add()([X,X_copy])
    X = Activation('relu')(X)
    return X

In [26]:
# function to upscale and concatenate the values passsed
def upsample_concat(x, skip):
    x = UpSampling2D((2,2))(x)
    merge = Concatenate()([x, skip])
    return merge

In [27]:
input_shape = (256,256,3)

# Input tensor shape
X_input = Input(input_shape)

# Stage 1
conv1_in = Conv2D(16,3,activation= 'relu', padding = 'same', kernel_initializer ='he_normal')(X_input)
conv1_in = BatchNormalization()(conv1_in)
conv1_in = Conv2D(16,3,activation= 'relu', padding = 'same', kernel_initializer ='he_normal')(conv1_in)
conv1_in = BatchNormalization()(conv1_in)
pool_1 = MaxPool2D(pool_size = (2,2))(conv1_in)

# Stage 2
conv2_in = resblock(pool_1, 32)
pool_2 = MaxPool2D(pool_size = (2,2))(conv2_in)

# Stage 3
conv3_in = resblock(pool_2, 64)
pool_3 = MaxPool2D(pool_size = (2,2))(conv3_in)

# Stage 4
conv4_in = resblock(pool_3, 128)
pool_4 = MaxPool2D(pool_size = (2,2))(conv4_in)

# Stage 5 (Bottle Neck)
conv5_in = resblock(pool_4, 256)

# Upscale stage 1
up_1 = upsample_concat(conv5_in, conv4_in)
up_1 = resblock(up_1, 128)

# Upscale stage 2
up_2 = upsample_concat(up_1, conv3_in)
up_2 = resblock(up_2, 64)

# Upscale stage 3
up_3 = upsample_concat(up_2, conv2_in)
up_3 = resblock(up_3, 32)

# Upscale stage 4
up_4 = upsample_concat(up_3, conv1_in)
up_4 = resblock(up_4, 16)

# Final Output
output = Conv2D(1, (1,1), padding = "same", activation = "sigmoid")(up_4)

model_seg = Model(inputs = X_input, outputs = output )


In [28]:
model_seg.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_27 (Conv2D)             (None, 256, 256, 16  448         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_26 (BatchN  (None, 256, 256, 16  64         ['conv2d_27[0][0]']              
 ormalization)                  )                                                           

In [29]:
# Utilities file contains the code for custom loss function and custom data generator

from utilities import focal_tversky, tversky_loss, tversky

In [30]:
# Compile the model
adam = tf.keras.optimizers.Adam(lr = 0.05, epsilon = 0.1)
model_seg.compile(optimizer = adam, loss = focal_tversky, metrics = [tversky])

In [31]:
# use early stopping to exit training if validation loss is not decreasing even after certain epochs (patience)
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

# save the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath="ResUNet-weights.hdf5", verbose=1, save_best_only=True)

In [32]:
history = model_seg.fit(training_generator, epochs = 5, validation_data = validation_generator, callbacks = [checkpointer, earlystopping])

Epoch 1/5
72/72 [==============================] - ETA: 0s - loss: 0.8314 - tversky: 0.2167
Epoch 1: val_loss improved from inf to 0.81407, saving model to ResUNet-weights.hdf5
72/72 [==============================] - 18s 210ms/step - loss: 0.8314 - tversky: 0.2167 - val_loss: 0.8141 - val_tversky: 0.2398
Epoch 2/5
72/72 [==============================] - ETA: 0s - loss: 0.4967 - tversky: 0.6035
Epoch 2: val_loss improved from 0.81407 to 0.49576, saving model to ResUNet-weights.hdf5
72/72 [==============================] - 14s 189ms/step - loss: 0.4967 - tversky: 0.6035 - val_loss: 0.4958 - val_tversky: 0.6057
Epoch 3/5
72/72 [==============================] - ETA: 0s - loss: 0.3918 - tversky: 0.7115
Epoch 3: val_loss improved from 0.49576 to 0.41742, saving model to ResUNet-weights.hdf5
72/72 [==============================] - 15s 201ms/step - loss: 0.3918 - tversky: 0.7115 - val_loss: 0.4174 - val_tversky: 0.6871
Epoch 4/5
72/72 [==============================] - ETA: 0s - loss: 0.33

In [35]:
# save the model architecture to json file for future use

model_json = model_seg.to_json()
with open("ResUNet-model.json","w") as json_file:
  json_file.write(model_json)